<a href="https://colab.research.google.com/github/gregory-ch/demo-E_G-dynamic/blob/master/vocabulary_rank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install  wikidata
%pip install translators --upgrade
# %pip install termcolor
# %pip install ipywidgets

In [ ]:
import requests
import re
from wikidata.client import Client
API_ENDPOINT = "https://en.wiktionary.org/w/api.php"
client = Client()

In [ ]:
def get_data(query):
    params = {
        'action': 'query',
        'format': 'json',
        'prop':'revisions',
        'rvprop':'content',
        'formatversion':'2',
        'pageids': query,
    }
    r = requests.get(API_ENDPOINT, params=params)
    return r.json()

def WikiParser(query):
  data = get_data(query)
  return re.findall('\[\[(.*?)\]\]', data['query']['pages'][0]['revisions'][0]['content'])

In [ ]:
first_10k = WikiParser('192306')[1:100]
# remove [1:100] for full functionality, here just an exammple (уберите [1:100] для того тчобы получить полный список, ограничение в 100 стоит для того чтобы освоится с функционалом)

In [ ]:
from google.colab import drive
drive.mount('drive')
import translators as ts
from tqdm import tqdm
from time import sleep
import pandas as pd
def translate_to_df(df, indexes):
  matches = tqdm(indexes, ncols=80, ascii=True, desc='Total')
  list_df = []
  for match in matches:
    try:
      list_df.append([df[match], ts.google(df[match] , to_language = 'ru')])
    except Exception as e: 
      print(e)
      list_df.append([df[match], None])
  return pd.DataFrame( columns = ['word', 'translation'],data = list_df )
ids = range(len(first_10k))
# ids = find_all_blank(first10kdf, ['word','is_rubbish','known','passive','active'])
first10kdf = translate_to_df(first_10k, ids)
first10kdf.to_csv('/content/drive/MyDrive/freq_list_10k3.csv',sep=',', header = True)

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import clear_output
first10kdf = pd.read_csv('/content/drive/MyDrive/freq_list_10k3.csv', index_col=0)
# first10kdf = first10kdf.reindex(first10kdf.columns.tolist() + ['is_rubbish','known','passive','active'], axis=1)

In [ ]:
def find_all_blank(df, keys):
  target_df=df[df.columns.difference(keys)]
  return list(pd.isnull(target_df).all(1).to_numpy().nonzero()[0])

# find_all_blank(first10kdf, ['word','is_rubbish','known','passive','active'])

In [ ]:
# %%capture --no-display
import ipywidgets as widgets
from termcolor import colored
inpit_area = widgets.Text(value='first10kdf', placeholder='Type something', description='name of df:',disabled=False)
strt_btn = widgets.Button(description='start')
put_to_rubbish_btn = widgets.Button(description='rubbish')
put_to_known_btn = widgets.Button(description='known')
put_to_active_btn = widgets.Button(description='active')
put_to_passive_btn = widgets.Button(description='passive')
end_btn = widgets.Button(description='quit&save')

df_towork=""
def strt_btn_eventhandler(obj):
    global df_towork 
    df_towork = globals()[inpit_area.value]
    numb = min(find_all_blank(df_towork, ['word','translation']))
    output = str('curent position "'+ str(df_towork.index[numb])+ '" word: "'+
                 colored(str(df_towork.iloc[numb].word),"green")+'" rus: "'+
    str(df_towork.iloc[numb].translation)+'"')
    print("\r", colored(output, attrs=["bold"]), end="")

strt_btn.on_click(strt_btn_eventhandler)

def end_save(obj):
    global df_towork 
    df_towork.to_csv(r'/content/drive/MyDrive/freq_list_10k3.csv',sep=',', header = True)
    clear_output()

end_btn.on_click(end_save)

def put_to_rubbish_btn_eventhandler(obj):
    global df_towork 
    df_towork=df_towork
    numb = min(find_all_blank(df_towork, ['word','translation']))
    df_towork.loc[numb,'is_rubbish']= 1
    output = str('curent position "'+ str(df_towork.index[numb+1])+ '" word: "'+
                 colored(str(df_towork.iloc[numb+1].word),"green")+'" rus: "'+
    str(df_towork.iloc[numb+1].translation)+'"')
    print("\r", colored(output, attrs=["bold"]), end="")

put_to_rubbish_btn.on_click(put_to_rubbish_btn_eventhandler)

def put_to_known_btn_eventhandler(obj):
    global df_towork 
    df_towork=df_towork
    numb = min(find_all_blank(df_towork, ['word','translation']))
    df_towork.loc[numb,'known']= 1
    output = str('curent position "'+ str(df_towork.index[numb+1])+ '" word: "'+
                 colored(str(df_towork.iloc[numb+1].word),"green")+'" rus: "'+
    str(df_towork.iloc[numb+1].translation)+'"')
    print("\r", colored(output, attrs=["bold"]), end="")

put_to_known_btn.on_click(put_to_known_btn_eventhandler)

def put_to_active_btn_eventhandler(obj):
    global df_towork 
    df_towork=df_towork
    numb = min(find_all_blank(df_towork, ['word','translation']))
    df_towork.loc[numb,'active']= 1
    output = str('curent position "'+ str(df_towork.index[numb+1])+ '" word: "'+
                 colored(str(df_towork.iloc[numb+1].word),"green")+'" rus: "'+
    str(df_towork.iloc[numb+1].translation)+'"')
    print("\r", colored(output, attrs=["bold"]), end="")

put_to_active_btn.on_click(put_to_active_btn_eventhandler)

def put_to_passive_btn_eventhandler(obj):
    global df_towork 
    df_towork=df_towork
    numb = min(find_all_blank(df_towork, ['word','translation']))
    df_towork.loc[numb,'passive']= 1
    output = str('curent position "'+ str(df_towork.index[numb+1])+ '" word: "'+
                 colored(str(df_towork.iloc[numb+1].word),"green")+'" rus: "'+
    str(df_towork.iloc[numb+1].translation)+'"')
    print("\r", colored(output, attrs=["bold"]), end="")

put_to_passive_btn.on_click(put_to_passive_btn_eventhandler)

display(widgets.HBox(children=[strt_btn,inpit_area,end_btn]),widgets.HBox(children=[put_to_rubbish_btn,
                                                                                    put_to_active_btn,put_to_passive_btn,
                                                                                    put_to_known_btn]))